## Step 01. Installing CrewAI and Tools

To install crewAI, we need to have Python >=3.10 and <=3.13 installed on your system:

In [1]:
pip install crewai

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install 'crewai[tools]'

Note: you may need to restart the kernel to use updated packages.


In [3]:
import warnings
warnings.filterwarnings('ignore')

## Step 02. Importing Libraries, API and LLMs

In [4]:
from crewai import Agent, Task, Crew

In [5]:
import os

openai_api_key = os.getenv("OPENAI_API_KEY")
if not openai_api_key:
    raise ValueError("OPENAI_API_KEY environment variable is not set.")
os.environ["OPENAI_API_KEY"] = openai_api_key
os.environ["OPENAI_MODEL_NAME"] = "gpt-3.5-turbo"

## Step 03. Creating Agent (with role, focus, cooperation)

In [6]:
customer_support_agent = Agent(
    role="Senior Customer Support Representative",
    goal="Be the most friendly and helpful "
        "customer support representative in your team",
    backstory=(
		"You work at crewAI (https://crewai.com) and "
        " are now working on providing customer"
		"support to {customer}, a super important customer "
        " for your company."
		"You need to make sure that you provide the best support!"
		"Make sure to provide full complete answers, "
        " and make no assumptions."
	),
    allow_delegration=False, 
    verbose=True
)

In [7]:
quality_assurance_agent = Agent(
    role="Support Quality Assurance Specialist",
    goal="Get recognition for providing the "
    "best support quality assurance in your team",
    backstory=(
		"You work at crewAI (https://crewai.com) and "
        "are now working with your team "
		"on a request from {customer} ensuring that "
        "the support representative is "
		"providing the best support possible.\n"
		"You need to make sure that the support representative "
        "is providing full"
		"complete answers, and make no assumptions."
	),
    verbose=True
)

## Step 04. Forming Task with Tools, Gaurdrails and Agents

In [8]:
from crewai_tools import SerperDevTool, ScrapeWebsiteTool, WebsiteSearchTool

* **SerperDevTool**: Do Google Search and get relevant search results
* **ScrapeWebsiteTool**: Web scraping and data collection
* **WebsiteSearchTool**: semantic searches within website content.

In [9]:
docs_scrape_tool = ScrapeWebsiteTool(
    website_url="https://docs.crewai.com/how-to/Creating-a-Crew-and-kick-it-off/"
)

In [10]:
inquiry_resolution = Task(
    description=(
        "{customer} just reached out with a super important ask:\n"
	    "{inquiry}\n\n"
        "{person} from {customer} is the one that reached out. "
		"Make sure to use everything you know "
        "to provide the best support possible."
		"You must strive to provide a complete "
        "and accurate response to the customer's inquiry."
    ),
    expected_output=(
	    "A detailed, informative response to the "
        "customer's inquiry that addresses "
        "all aspects of their question.\n"
        "The response should include references "
        "to everything you used to find the answer, "
        "including external data or solutions. "
        "Ensure the answer is complete, "
		"leaving no questions unanswered, and maintain a helpful and friendly "
		"tone throughout."
    ),
    agent=customer_support_agent,
    tools=[docs_scrape_tool]
)

In [11]:
quality_assurance_review = Task(
    description=(
        "Review the response drafted by the Senior Support Representative for {customer}'s inquiry. "
        "Ensure that the answer is comprehensive, accurate, and adheres to the "
		"high-quality standards expected for customer support.\n"
        "Verify that all parts of the customer's inquiry "
        "have been addressed "
		"thoroughly, with a helpful and friendly tone.\n"
        "Check for references and sources used to "
        " find the information, "
		"ensuring the response is well-supported and "
        "leaves no questions unanswered."
    ),
    expected_output=(
        "A final, detailed, and informative response "
        "ready to be sent to the customer.\n"
        "This response should fully address the "
        "customer's inquiry, incorporating all "
		"relevant feedback and improvements.\n"
		"Don't be too formal, we are a chill and cool company "
	    "but maintain a professional and friendly tone throughout."
    ),
    agent=quality_assurance_agent,
)

## Step 05. Creating Crew (with Memory)

#### Memory
- Setting `memory=True` when putting the crew together enables all kinds of memory.

In [12]:
crew = Crew(
    agents=[customer_support_agent, quality_assurance_agent],
    tasks=[inquiry_resolution, quality_assurance_review],
    verbose=True,
    memory=True
)

In [13]:
inputs = {
    "customer": "DeepLearningAI",
    "person": "Andrew Ng",
    "inquiry": "I need help with setting up a Crew "
               "and kicking it off, specifically "
               "how can I add memory to my crew? "
               "Can you provide guidance?"
}
result = crew.kickoff(inputs=inputs)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: cfbec680-65a8-4875-a195-b5032022df3b                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Senior Customer Support Representative                                                                  │
│                                                                                                                 │
│  Task: DeepLearningAI just reached out with a super important ask:                                              │
│  I need help with setting up a Crew and kicking it off, specifically how can I add memory to my crew? Can you   │
│  provide guidance?                                                                                              │
│                                                                                                                 │
│  Andrew Ng from DeepLearningAI is the one that reached out. Make sure to use everything you know to provide     │
│  the best support possible.You must strive to provide a complete and accurate response to the customer's        │
│  inquiry.                                                                                                       │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Senior Customer Support Representative                                                                  │
│                                                                                                                 │
│  Thought: I need to provide Andrew Ng from DeepLearningAI with detailed guidance on how to add memory to their  │
│  Crew while setting it up and kicking it off. To ensure I provide the most accurate and comprehensive           │
│  response, I should use the available tools, starting with reading the content from the CrewAI website.         │
│                                                                                                                 │
│  Using Tool: Read website content                                                                               │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...


[2025-12-25 16:08:46][ERROR]: Failed to add to long term memory: Failed to convert text into a Pydantic model due to error: Error code: 400 - {'error': {'message': "Invalid parameter: 'response_format' of type 'json_schema' is not supported with this model. Learn more about supported models at the Structured Outputs guide: https://platform.openai.com/docs/guides/structured-outputs", 'type': 'invalid_request_error', 'param': None, 'code': None}}


╭─────────────────────────────────────────────────── LLM Error ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  ❌ LLM Call Failed                                                                                             │
│  Error: OpenAI API call failed: Error code: 400 - {'error': {'message': "Invalid parameter: 'response_format'   │
│  of type 'json_schema' is not supported with this model. Learn more about supported models at the Structured    │
│  Outputs guide: https://platform.openai.com/docs/guides/structured-outputs", 'type': 'invalid_request_error',   │
│  'param': None, 'code': None}}                                                                                  │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: d6e7a456-23bb-4255-a8cc-f3ab99497dd2                                                                     │
│  Agent: Support Quality Assurance Specialist                                                                    │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Tracing Status ─────────────────────────────────────────────────╮
│                                                                                                                 │
│  Info: Tracing is disabled.                                                                                     │
│                                                                                                                 │
│  To enable tracing, do any one of these:                                                                        │
│  • Set tracing=True in your Crew/Flow code                                                                      │
│  • Set CREWAI_TRACING_ENABLED=true in your project's .env file                                                  │
│  • Run: crewai traces enable                                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [14]:
from IPython.display import Markdown
Markdown(result)

TypeError: Markdown expects text, not CrewOutput(raw="Hello,\n\nThank you for reaching out with your inquiry about adding memory to your Crew while setting it up and kicking it off. I understand the significance of this process, and I'm here to provide you with a comprehensive guide to assist you every step of the way.\n\nTo effectively incorporate memory into your Crew, you can follow the detailed guidance provided in the CrewAI Documentation. Here are the specific steps you should take to ensure a successful setup and launch:\n\n1. Begin by exploring the fundamental CrewAI concepts, architecture, and the potential of building agents, crews, and flows. This foundational knowledge will be essential in understanding how memory can be integrated into your Crew seamlessly.\n\n2. Install CrewAI via uv, configure the necessary API keys, and set up the CLI for local development. This setup is crucial as it will grant you access to essential tools and features required to work efficiently wit